In [2]:
def map_actual_value(value, min_value, max_value):
    if max_value == min_value:
        return 0
    else:
        return (value - min_value) / (max_value - min_value)

In [3]:
import csv

# Read the CSV data
with open('../static/data/netCongestion.csv') as f:
    reader = csv.DictReader(f)
    data = list(reader)

# Convert the Actual values to floats
for item in data:
    item['Actual'] = float(item['Actual'])

# Find the min and max Actual values
min_actual = min(item['Actual'] for item in data)
max_actual = max(item['Actual'] for item in data)

# Map the Actual values to a range of 0 to 1
for item in data:
    item['Mapped'] = map_actual_value(item['Actual'], min_actual, max_actual)


In [4]:
data

[{'Date': '2008-05-17', 'Actual': 879.0, 'Mapped': 0.22894168466522677},
 {'Date': '2008-05-18', 'Actual': 839.0, 'Mapped': 0.14254859611231102},
 {'Date': '2008-05-19', 'Actual': 886.0, 'Mapped': 0.24406047516198703},
 {'Date': '2008-05-20', 'Actual': 911.0, 'Mapped': 0.2980561555075594},
 {'Date': '2008-05-21', 'Actual': 918.0, 'Mapped': 0.31317494600431967},
 {'Date': '2008-05-22', 'Actual': 851.0, 'Mapped': 0.16846652267818574},
 {'Date': '2008-05-23', 'Actual': 831.0, 'Mapped': 0.12526997840172785},
 {'Date': '2008-05-24', 'Actual': 880.0, 'Mapped': 0.23110151187904968},
 {'Date': '2008-05-25', 'Actual': 933.0, 'Mapped': 0.34557235421166305},
 {'Date': '2008-05-26', 'Actual': 992.0, 'Mapped': 0.47300215982721383},
 {'Date': '2008-05-27', 'Actual': 865.0, 'Mapped': 0.19870410367170627},
 {'Date': '2008-05-28', 'Actual': 859.0, 'Mapped': 0.1857451403887689},
 {'Date': '2008-05-29', 'Actual': 853.0, 'Mapped': 0.17278617710583152},
 {'Date': '2008-05-30', 'Actual': 773.0, 'Mapped': 0.

In [8]:
with open('../static/data/netCongestion.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['Date', 'Actual', 'Mapped'])
    writer.writeheader()
    writer.writerows(data)

In [18]:
import pandas as pd
import numpy as np

# CSV 파일에서 데이터 불러오기
df = pd.read_csv('../static/data/netCongestion.csv', parse_dates=['Date'])

# Date 컬럼을 Datetime 형식으로 변경
df['Date'] = pd.to_datetime(df['Date'])

# Time 컬럼 추가
df['Time'] = pd.date_range('00:00', periods=len(df), freq='H').hour

# 출근시간과 퇴근시간 설정
start_time = 9
end_time = 18

# 출근시간 전과 퇴근시간 후에는 Actual 값을 0으로 설정
df.loc[df['Time'] < start_time, 'Actual'] = 0
df.loc[df['Time'] > end_time, 'Actual'] = 0

# 시간별 Actual 값을 다르게 만들기
for i in range(start_time, end_time+1):
    if i % 2 == 0:
        df.loc[df['Time'] == i, 'Actual'] += 100
    else:
        df.loc[df['Time'] == i, 'Actual'] -= 50

# Actual 값이 음수인 경우 0으로 설정
df.loc[df['Actual'] < 0, 'Actual'] = 0

# Time 컬럼을 0에서 24의 정수로 변경
df['Time'] = df['Time'].astype(int)

# 결과 출력
print(df.to_csv(index=False))

Date,Actual,Time
2008-05-17,0,0
2008-05-18,0,1
2008-05-19,0,2
2008-05-20,0,3
2008-05-21,0,4
2008-05-22,0,5
2008-05-23,0,6
2008-05-24,0,7
2008-05-25,0,8
2008-05-26,942,9
2008-05-27,965,10
2008-05-28,809,11
2008-05-29,953,12
2008-05-30,723,13
2008-05-31,939,14
2008-06-01,848,15
2008-06-02,989,16
2008-06-03,791,17
2008-06-04,942,18
2008-06-05,0,19
2008-06-06,0,20
2008-06-07,0,21
2008-06-08,0,22
2008-06-09,0,23



In [2]:
import pandas as pd
import numpy as np

# 데이터 불러오기
df = pd.read_csv('../static/data/netCongestion.csv')

# Actual 값 정규화
df['Actual_norm'] = (df['Actual'] - df.groupby('Date')['Actual'].transform('min')) / (df.groupby('Date')['Actual'].transform('max') - df.groupby('Date')['Actual'].transform('min'))

# Actual 값 조정
df['Actual_adj'] = df['Actual_norm'] * (df.groupby('Date')['Actual_norm'].transform('max') - df.groupby('Date')['Actual_norm'].transform('min')) + df.groupby('Date')['Actual_norm'].transform('min')

# 결과 출력
print(df)


          Date  Actual  Actual_norm  Actual_adj
0   2008-05-17     879          NaN         NaN
1   2008-05-18     839          NaN         NaN
2   2008-05-19     886          NaN         NaN
3   2008-05-20     911          NaN         NaN
4   2008-05-21     918          NaN         NaN
5   2008-05-22     851          NaN         NaN
6   2008-05-23     831          NaN         NaN
7   2008-05-24     880          NaN         NaN
8   2008-05-25     933          NaN         NaN
9   2008-05-26     992          NaN         NaN
10  2008-05-27     865          NaN         NaN
11  2008-05-28     859          NaN         NaN
12  2008-05-29     853          NaN         NaN
13  2008-05-30     773          NaN         NaN
14  2008-05-31     839          NaN         NaN
15  2008-06-01     898          NaN         NaN
16  2008-06-02     889          NaN         NaN
17  2008-06-03     841          NaN         NaN
18  2008-06-04     842          NaN         NaN
19  2008-06-05     851          NaN     